In [2]:
# import libraries
import json
import requests
import pandas as pd
import csv
import psycopg2

In [5]:
# extraction layer
url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"200000"}

headers = {
	"x-rapidapi-key": "57015117a6msh483a52f18858c6ap1ce26bjsnf768f7285631",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
data = response.json()

filename = 'propertyRecords.json'
with open(filename, 'w') as file:
    json.dump(data, file, indent=5)



In [6]:
property_json_df = pd.read_json('propertyRecords.json')

In [7]:
property_json_df.head()

,bathrooms,bedrooms,squareFootage,county,propertyType,addressLine1,addressLine2,city,state,zipCode,...,features,taxAssessment,ownerOccupied,owner,id,longitude,latitude,zoning,propertyTaxes,yearBuilt
0,2.0,2.0,1847.0,Westchester,Condo,1 Scarsdale Rd,Apt 619,Tuckahoe,NY,10707,...,{'architectureType': 'Condo / Apartment'},"{'2022': {'value': 3840, 'land': 1050, 'improv...",0.0,"{'names': ['Mary A Pipolo'], 'mailingAddress':...","1-Scarsdale-Rd,-Apt-619,-Tuckahoe,-NY-10707",-73.828732,40.951534,NaN,NaN,NaN
1,3.0,6.0,3326.0,Pasco,Land,8830 Parsons Hill Blvd,NaN,Wesley Chapel,FL,33545,...,{},{'2022': {'value': 50728}},NaN,"{'names': ['MUSTAFA ALI', 'SOLIMAN NADA'], 'ma...","8830-Parsons-Hill-Blvd,-Wesley-Chapel,-FL-33545",-82.292526,28.283342,MPUD,{'2022': {'total': 2955}},NaN
2,2.0,4.0,3263.0,Brevard,Single Family,4145 Fenrose Cir,NaN,Melbourne,FL,32940,...,"{'cooling': True, 'coolingType': 'Central', 'e...",{'2022': {'value': 189890}},1.0,"{'names': ['ANNE R SIMONS'], 'mailingAddress':...","4145-Fenrose-Cir,-Melbourne,-FL-32940",-80.693933,28.187640,RU-1-7,{'2022': {'total': 2496}},2003.0
3,2.0,3.0,1604.0,Riverside,Single Family,24873 Butterchurn Rd,NaN,Wildomar,CA,92595,...,"{'cooling': True, 'coolingType': 'Central', 'f...","{'2022': {'value': 189366, 'land': 71010, 'imp...",1.0,"{'names': ['Anton L Felando', 'NATALIE L FELAN...","24873-Butterchurn-Rd,-Wildomar,-CA-92595",-117.226109,33.626908,RT,"{'2022': {'total': 2823}, '2023': {'total': 28...",2005.0
4,2.5,3.0,2694.0,Clark,Single Family,71 Sahalee Dr,NaN,Las Vegas,NV,89148,...,"{'architectureType': 'Single Story', 'cooling'...","{'2020': {'value': 136922, 'land': 41160, 'imp...",1.0,"{'names': ['WATSON SUSAN G TRUST'], 'mailingAd...","71-Sahalee-Dr,-Las-Vegas,-NV-89148",-115.287114,36.047489,NaN,"{'2019': {'total': 3566}, '2021': {'total': 37...",1998.0


In [103]:
property_json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bathrooms         500 non-null    float64
 1   bedrooms          500 non-null    float64
 2   squareFootage     500 non-null    float64
 3   county            500 non-null    object 
 4   propertyType      500 non-null    object 
 5   addressLine1      500 non-null    object 
 6   addressLine2      500 non-null    object 
 7   city              500 non-null    object 
 8   state             500 non-null    object 
 9   zipCode           500 non-null    int64  
 10  formattedAddress  500 non-null    object 
 11  assessorID        500 non-null    object 
 12  legalDescription  500 non-null    object 
 13  subdivision       500 non-null    object 
 14  lotSize           500 non-null    float64
 15  lastSalePrice     500 non-null    float64
 16  lastSaleDate      500 non-null    object 
 1

In [104]:
# Transformatio Layer
# 1st step, convert dictionary column to string
property_json_df['features'] = property_json_df['features'].apply(json.dumps)

# 2nd step: replace NaN values with appropriate defaults or remove row/columns as neccessary
property_json_df.fillna({
    'assessorID': 'Unknown',
    'legalDescription': 'Not Available',
    'squareFootage': 0,
    'subdivision': 'Not Available',
    'yearBuilt': 0,
    'bathrooms': 0,
    'lotSize': 0,
    'propertyType': 'Unknown',
    'lastSalePrice': 0,
    'lastSaleDate': 'Not Available',
    'features': 'None',
    'taxAssessment': 'Not Available',
    'owner': 'Unknown',
    'propertyTaxes': 'Not Available',
    'bedrooms': 0,
    'ownerOccupied': 0,
    'zoning': 'Unknown',
    'addressLine2': 'Not Available'
}, inplace=True)

In [105]:
property_json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bathrooms         500 non-null    float64
 1   bedrooms          500 non-null    float64
 2   squareFootage     500 non-null    float64
 3   county            500 non-null    object 
 4   propertyType      500 non-null    object 
 5   addressLine1      500 non-null    object 
 6   addressLine2      500 non-null    object 
 7   city              500 non-null    object 
 8   state             500 non-null    object 
 9   zipCode           500 non-null    int64  
 10  formattedAddress  500 non-null    object 
 11  assessorID        500 non-null    object 
 12  legalDescription  500 non-null    object 
 13  subdivision       500 non-null    object 
 14  lotSize           500 non-null    float64
 15  lastSalePrice     500 non-null    float64
 16  lastSaleDate      500 non-null    object 
 1

In [106]:
# create the fact table
fact_columns = ['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress', 'squareFootage', 'yearBuilt', 'bathrooms', 'bedrooms', 'lotSize', 'propertyType', 'longitude', 'latitude']
fact_table = property_json_df[fact_columns]

In [107]:
# create location dimension
location_dim = property_json_df[['addressLine1','city','state', 'zipCode', 'county', 'longitude', 'latitude']].drop_duplicates().reset_index(drop=True)
location_dim.index.name = 'location_id'

In [108]:
location_dim

,addressLine1,city,state,zipCode,county,longitude,latitude
location_id,,,,,,,
0,1 Scarsdale Rd,Tuckahoe,NY,10707,Westchester,-73.828732,40.951534
1,8830 Parsons Hill Blvd,Wesley Chapel,FL,33545,Pasco,-82.292526,28.283342
2,4145 Fenrose Cir,Melbourne,FL,32940,Brevard,-80.693933,28.187640
3,24873 Butterchurn Rd,Wildomar,CA,92595,Riverside,-117.226109,33.626908
4,71 Sahalee Dr,Las Vegas,NV,89148,Clark,-115.287114,36.047489
...,...,...,...,...,...,...,...
495,1932 W Toledo Ct,Broken Arrow,OK,74012,Tulsa,-95.812956,36.034246
496,3079 Tamarack Ave,Slayton,MN,56172,Murray,-95.743879,43.981577
497,7970 W R Ave,Kalamazoo,MI,49009,Kalamazoo,-85.703502,42.188660


In [109]:
# create sales dimension
sales_dim = property_json_df[['lastSalePrice', 'lastSaleDate']].drop_duplicates().reset_index(drop=True)
sales_dim.index.name = 'sales_id'

In [110]:
sales_dim

,lastSalePrice,lastSaleDate
sales_id,,
0,547180.0,2010-01-28T00:00:00.000Z
1,590000.0,2022-07-22T00:00:00.000Z
2,226000.0,2003-07-24T00:00:00.000Z
3,160000.0,2011-10-28T00:00:00.000Z
4,225000.0,1998-02-11T00:00:00.000Z
...,...,...
304,0.0,2017-12-27T00:00:00.000Z
305,89000.0,2003-03-05T00:00:00.000Z
306,150000.0,2004-08-01T00:00:00.000Z


In [111]:
# create property features dimension
features_dim = property_json_df[['features', 'propertyType', 'zoning']].drop_duplicates().reset_index(drop=True)
features_dim.index.name = 'features_id'

In [112]:
features_dim

,features,propertyType,zoning
features_id,,,
0,"""\""\\\""\\\\\\\""\\\\\\\\\\\\\\\""\\\\\\\\\\\\\\\...",Condo,Unknown
1,"""\""\\\""\\\\\\\""\\\\\\\\\\\\\\\""\\\\\\\\\\\\\\\...",Land,MPUD
2,"""\""\\\""\\\\\\\""\\\\\\\\\\\\\\\""\\\\\\\\\\\\\\\...",Single Family,RU-1-7
3,"""\""\\\""\\\\\\\""\\\\\\\\\\\\\\\""\\\\\\\\\\\\\\\...",Single Family,RT
4,"""\""\\\""\\\\\\\""\\\\\\\\\\\\\\\""\\\\\\\\\\\\\\\...",Single Family,Unknown
...,...,...,...
345,"""\""\\\""\\\\\\\""\\\\\\\\\\\\\\\""\\\\\\\\\\\\\\\...",Single Family,Unknown
346,"""\""\\\""\\\\\\\""\\\\\\\\\\\\\\\""\\\\\\\\\\\\\\\...",Single Family,Unknown
347,"""\""\\\""\\\\\\\""\\\\\\\\\\\\\\\""\\\\\\\\\\\\\\\...",Single Family,R3


In [113]:
fact_table.to_csv('property_fact.csv', index=False)
location_dim.to_csv('location_dimension.csv', index=True)
sales_dim.to_csv('sales_dimension.csv', index=True)
features_dim.to_csv('features_dimension.csv', index=True)

In [114]:
# loading layer
# develop a function to connect to pgadmin
def get_db_connection():
    connection = psycopg2.connect(host='127.0.0.1', database='postgres', user='isaacoduh', password='root')
    return connection

# connect to sql database
conn = get_db_connection()

In [115]:
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''
        CREATE SCHEMA IF NOT EXISTS propco;

        DROP TABLE IF EXISTS propco.fact_table;
        DROP TABLE IF EXISTS propco.location_dim;
        DROP TABLE IF EXISTS propco.sales_dim;
        DROP TABLE IF EXISTS propco.features_dim;

        CREATE TABLE propco.fact_table (
            addressLine1 VARCHAR(255),
            city VARCHAR(100),
            state VARCHAR(50),
            zipCode INTEGER,
            formattedAddress VARCHAR(255),
            squareFootage FLOAT,
            yearBuilt FLOAT,
            bathrooms FLOAT,
            bedrooms FLOAT,
            lotSize FLOAT,
            propertyType VARCHAR(100),
            longitude FLOAT,
            latitude FLOAT
        );

        CREATE TABLE propco.location_dim (
            location_id SERIAL PRIMARY KEY,
            addressLine1 VARCHAR(255),
            city VARCHAR(100),
            state VARCHAR(50),
            zipCode INTEGER,
            county VARCHAR(100),
            longitude FLOAT,
            latitude FLOAT
        );

        CREATE TABLE propco.sales_dim(
            sales_id SERIAL PRIMARY KEY,
            lastSalePrice FLOAT,
            lastSaleDate DATE
        );

        CREATE TABLE propco.features_dim(
            features_id SERIAL PRIMARY KEY,
            features TEXT,
            propertyType VARCHAR(100),
            zoning VARCHAR
        );
    '''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

create_tables()

In [116]:
# create function to load data to tables
def load_data_from_csv_to_tables(csv_path, table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader) # skip the header
        for row in reader:
            placeholders = ", ".join(['%s'] * len(row))
            query = f'INSERT INTO {table_name} VALUES ({placeholders});'
            cursor.execute(query, row)
    conn.commit()
    cursor.close()
    conn.close()

In [117]:
# fact table
fact_csv_path = r'/Users/isaacoduh/DevSandbox/data_portfolio/DE/propco_api_to_postgres_etl/property_fact.csv'
load_data_from_csv_to_tables(fact_csv_path, 'propco.fact_table')

In [118]:
# locatioin dimension table
fact_csv_path = r'/Users/isaacoduh/DevSandbox/data_portfolio/DE/propco_api_to_postgres_etl/location_dimension.csv'
load_data_from_csv_to_tables(fact_csv_path, 'propco.location_dim')

In [119]:
# features dimension table
fact_csv_path = r'/Users/isaacoduh/DevSandbox/data_portfolio/DE/propco_api_to_postgres_etl/features_dimension.csv'
load_data_from_csv_to_tables(fact_csv_path, 'propco.features_dim')

In [123]:
# create function to load data to tables
def load_data_from_csv_to_sales_tables(csv_path, table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        
        next(reader) # skip the header row
        for row in reader:

            # convert empty strings (or 'Not available') in date column
            row = [None if (cell == '' or cell =='Not Available' or cell =='Not available') and col_name == 'lastSaleDate' else cell for cell, col_name in zip(row, ['sales_id', 'lastSalePrice', 'lastSaleDate'])]
            placeholders = ", ".join(['%s'] * len(row))
            query = f'INSERT INTO {table_name} VALUES ({placeholders});'
            cursor.execute(query, row)
            

sales_dim_columns = ['sales_id', 'lastSalePrice', 'lastSaleDate']

fact_csv_path = r'/Users/isaacoduh/DevSandbox/data_portfolio/DE/propco_api_to_postgres_etl/sales_dimension.csv'
load_data_from_csv_to_sales_tables(fact_csv_path, 'propco.sales_dim')